## DATA PREPARATION

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from math import radians, cos, sin, asin, sqrt
from datetime import datetime
from matplotlib import pyplot as plt
import holidays

In [2]:
path = '/Users/tolgacan/taxi/cabspottingdata/'
all_files = listdir(path)

#### We have data for 538 cabs and collected over a month. Each file have 4 columns ('latitude,longitude,occupation,time)

## READ DATA

### USER DEFINED FUNCTIONS

In [3]:
def read_data(filename):
    initial_df = pd.read_csv(path+'/'+filename,sep=' ',header=None)
    initial_df.columns = ['latitude','longitude','occupation','time']
    taxi_id = filename.split('.')[0]
    initial_df['taxi_id'] = taxi_id
    return initial_df

In [4]:
#creating shifted columns to use for different purposes
def generate_lag_columns(df):
    df['next_latitude'] = df.latitude.shift(-1)
    df['next_longitude'] = df.longitude.shift(-1)
    df['next_occupation_status']=df.occupation.shift(-1)
    df['previous_occupation_status']=df.occupation.shift(1)
    df['next_time'] = df.time.shift(-1)
    return df

In [5]:
##https://www.geeksforgeeks.org/program-distance-two-points-earth/
def distance(lat1, lat2, lon1, lon2):
     
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))    
    r = 3959

    return(c * r)

In [6]:
#function that calculate distance in miles between current and next location
def calculate_distance(df):
    df['distance_miles'] = [distance(x[0],x[5],x[1],x[6]) for x in df.values ]
    return df

I have identified target variable as following:
when taxi status is "0", I found the next line which occupation status is "1" and write latitude and longitude information as target variable.
My assumption here, taxi is occupied when we see occupation 1 not before the line. it is same for 0 status.

In [7]:
def calculation_target_variable(df):
    #find indexes which occupation status changes from 1 to 0 and 0 to 1.
    status_df_from_0_to_1 = df[(df['occupation']==1)& (df['previous_occupation_status']==0)]
    status_df_from_1_to_0 = df[(df['occupation']==0)& (df['previous_occupation_status']==1)]
    df['target_column'] = ''
    if df.loc[0,'occupation'] ==0: #if file is started with the occupation status 0
        for idx in range(len(status_df_from_0_to_1)):
            idx_position0 = status_df_from_1_to_0.index[idx-1] #find index when it turns to 0
            idx_position1 = status_df_from_0_to_1.index[idx] #find index when it turns to 1
            if idx==0:
                #write the latitude and longitude together to target column when returns to 1
                df.loc[:idx_position1-1,'target_column'] = str(round(status_df_from_0_to_1.loc[idx_position1,'latitude'],2))+str(round(status_df_from_0_to_1.loc[idx_position1,'longitude'],2))
            else:
                df.loc[idx_position0-1:idx_position1-1,'target_column'] =\
                str(round(status_df_from_0_to_1.loc[idx_position1,'latitude'],2))+str(round(status_df_from_0_to_1.loc[idx_position1,'longitude'],2))
    else: #if file starts with occupation status 1
        for idx in range(len(status_df_from_0_to_1)):
            if idx != 0:
                idx_position0 = status_df_from_1_to_0.index[idx-1]
                idx_position1 = status_df_from_0_to_1.index[idx-1]
                df.loc[idx_position0-1:idx_position1-1,'target_column'] =\
                str(round(status_df_from_0_to_1.loc[idx_position1,'latitude'],2))+str(round(status_df_from_0_to_1.loc[idx_position1,'longitude'],2))
    return df

In [8]:
#converting timestamp column to datetime format, also make some feature engineering like getting day_of_week,hour,holiday etc.
def generate_time_columns(df):
    df['time'] = [datetime.fromtimestamp(x) for x in df['time']]
    df['day_of_week'] = [x.weekday() for x in df['time']]
    df['hour'] = df['time'].dt.hour
    us_holidays = holidays.CountryHoliday('US', prov=None, state='CA')
    df['holiday'] = [1 if x in us_holidays else 0 for x in df['time']]
    return df

In [9]:
#reading all taxi cabs files
all_df = pd.DataFrame()
for file in all_files:
    df = read_data(file)
    df = generate_lag_columns(df)
    df = calculate_distance(df)
    df = calculation_target_variable(df)
    df = generate_time_columns(df)
    all_df = all_df.append(df)

In [10]:
all_df.shape

(11220058, 15)

In [11]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
all_df.describe()

,latitude,longitude,occupation,next_latitude,next_longitude,next_occupation_status,previous_occupation_status,next_time,distance_miles,day_of_week,hour,holiday
count,11220058.00,11220058.00,11220058.00,11219521.00,11219521.00,11219521.00,11219521.00,11219521.00,11219521.00,11220058.00,11220058.00,11220058.00
mean,37.76,-122.41,0.45,37.76,-122.41,0.45,0.45,1212036423.61,0.26,3.12,11.16,0.04
std,0.05,0.04,0.50,0.05,0.04,0.50,0.50,588542.74,1.41,2.09,7.21,0.19
min,32.87,-127.08,0.00,32.87,-127.08,0.00,0.00,1211018409.00,0.00,0.00,0.00,0.00
25%,37.76,-122.43,0.00,37.76,-122.43,0.00,0.00,1211522891.00,0.02,1.00,5.00,0.00
50%,37.78,-122.41,0.00,37.78,-122.41,0.00,0.00,1212043253.00,0.15,3.00,10.00,0.00
75%,37.79,-122.40,1.00,37.79,-122.40,1.00,1.00,1212548825.00,0.29,5.00,18.00,0.00
max,50.31,-115.56,1.00,50.31,-115.56,1.00,1.00,1213089934.00,877.17,6.00,23.00,1.00


### SAVE TO PICKLE FILE

In [12]:
all_df.to_pickle('all_df.pkl')